In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf


path = "./vet_dataset_cleaned/"

# The meaning of life
random.seed(42)  
np.random.seed(42)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import glob
import paths as paths


patient_paths = paths.get_patient_paths(path)
patient_paths.sort()

img_paths = [glob.glob(path + "/img/*") for path in patient_paths]
mask_paths = [glob.glob(path + "/mask/*") for path in patient_paths]

In [ ]:
valid = int(len(img_paths) * 0.15 // 1)
test = int(len(img_paths) * 0.1 // 1)
train = int(len(img_paths) - valid - test)

print(train, valid, test)

In [ ]:
train_inputs = paths.flatten_list(img_paths[0:train])
train_truths = paths.flatten_list(mask_paths[0:train])

train_inputs.sort()
train_truths.sort()

valid_inputs = paths.flatten_list(img_paths[train:train+valid])
valid_truths = paths.flatten_list(mask_paths[train:train+valid])

valid_inputs.sort()
valid_truths.sort()

test_inputs = paths.flatten_list(img_paths[train+valid:])
test_truths = paths.flatten_list(mask_paths[train+valid:])

test_inputs.sort()
test_truths.sort()

print(len(train_inputs))
print(len(valid_inputs))
print(len(test_inputs))

In [ ]:
def shuffle_together(inputs, truths):
    shuffle_together = list(zip(inputs, truths))
    random.shuffle(shuffle_together)
    inputs, truths = zip(*shuffle_together)
    return inputs, truths

In [ ]:
train_inputs, train_truths = shuffle_together(train_inputs, train_truths)
valid_inputs, valid_truths = shuffle_together(valid_inputs, valid_truths)

In [ ]:
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
import cv2
import skimage.transform


def gaussian_noise(img, mean=0, sigma=0.003):
    img = img.copy()
    noise = np.random.normal(mean, sigma, img.shape)
    mask_overflow_upper = img+noise >= 1.0
    mask_overflow_lower = img+noise < 0
    noise[mask_overflow_upper] = 1.0
    noise[mask_overflow_lower] = 0
    img = img + noise
    return img

def random_crop_resize(img, label, crop_size=492):
    size_img = img.shape
    size_label = label.shape
    crop_size = random.randint(crop_size, img.shape[0]-1)
    crop_size = (crop_size, crop_size)

    # "Crop size should be less than image size"
    assert crop_size[0] <= img.shape[0] and crop_size[1] <= img.shape[1]

    w, h = img.shape[:2]
    x, y = np.random.randint(h-crop_size[0]), np.random.randint(w-crop_size[1])

    img = img[y:y+crop_size[0], x:x+crop_size[1],:]
    img = skimage.transform.resize(img, size_img)

    label = label[y:y+crop_size[0], x:x+crop_size[1],:]
    label = skimage.transform.resize(label, size_label)
    return img, label

def affine_transform(image, label, alpha_affine=0.5, random_state=None):

    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)

    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    image = image[...,np.newaxis]
    label = cv2.warpAffine(label, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    return image, label


def data_augment(img, mask, chance=1):
    # flip l/r
    if random.uniform(0,1) < chance:
        img = cv2.flip( img, 1 )
        mask = cv2.flip( mask, 1 )
        if len(img.shape) == 2:
            img = img[...,np.newaxis]
        if len(mask.shape) == 2:
            mask = mask[...,np.newaxis]

    # random crop and resize
    if random.uniform(0,1) < chance:
        img, mask = random_crop_resize(img, mask)
        if len(mask.shape) == 2:
            label = label[...,np.newaxis]

    # random affine transformation
    if random.uniform(0,1) < chance:
        img, mask = affine_transform(img, mask, alpha_affine=20)
        if len(img.shape) == 2:
            img = img[...,np.newaxis]
        if len(mask.shape) == 2:
            mask = mask[...,np.newaxis]
            
    # random gaussian noise
    if random.uniform(0,1) < chance:
        img = gaussian_noise(img)
        
    return img, mask

def normalise(x, mean, std):
    return (x - mean) / std


def read_array_list(arr_path_list):
    return np.array([np.load(arr_path) for arr_path in arr_path_list])


def data_generator(input_paths, truth_paths, batch_size, mean, std, augment=False):
    batch_number = 0
    while batch_number < len(input_paths):
        if batch_number + batch_size <= len(input_paths):
            batch_input_paths = input_paths[batch_number: batch_number + batch_size]
            batch_truth_paths = truth_paths[batch_number: batch_number + batch_size]
        else:
            batch_input_paths = input_paths[batch_number:]
            batch_truth_paths = truth_paths[batch_number:]
        batch_number += batch_size
        
        batch_imgs = []
        batch_masks = []
        
        for x, y in zip(batch_input_paths, batch_truth_paths):
            x = np.load(x)
            y = np.load(y)
            if augment is True:
               
                
                x = normalise(x, mean, std)
                
                if random.uniform(0,1) < 0.5:
                    x, y = data_augment(x, y, chance=0.5)

            batch_imgs.append(x)
            batch_masks.append(y)
        
        yield np.array(batch_imgs, dtype=np.float32), np.array(batch_masks, dtype=np.float32)

In [ ]:
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
import cv2
import skimage.transform

class make_gen(tf.keras.utils.Sequence):
    def __init__( self, input_paths, label_paths, batch_size, training_mean, training_std, shuffle_on_end=True, augment=True):
        self.input_paths = input_paths
        self.label_paths = label_paths
        self.batch_size = batch_size
        self.training_mean = training_mean
        self.training_std = training_std
        self.shuffle_on_end = shuffle_on_end
        self.augment = augment

    def __len__(self):
        # number of batches per epoch
        return int(np.ceil(len(self.input_paths) / float(self.batch_size)))


    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        if self.shuffle_on_end == True:
            self.inputs, self.truths = self.suffle_together(self.inputs, self.truths)
            
    def shuffle_together(self, inputs, truths):
        shuffle_together = list(zip(inputs, truths))
        random.shuffle(shuffle_together)
        inputs, truths = zip(*shuffle_together)
        return inputs, truths
    
    
    def gaussian_noise(self, img, mean=0, sigma=0.003):
        img = img.copy()
        noise = np.random.normal(mean, sigma, img.shape)
        mask_overflow_upper = img+noise >= 1.0
        mask_overflow_lower = img+noise < 0
        noise[mask_overflow_upper] = 1.0
        noise[mask_overflow_lower] = 0
        img = img + noise
        return img

    def random_crop_resize(self, img, label, crop_size=492):
        size_img = img.shape
        size_label = label.shape
        crop_size = random.randint(crop_size, img.shape[0]-1)
        crop_size = (crop_size, crop_size)

        # "Crop size should be less than image size"
        assert crop_size[0] <= img.shape[0] and crop_size[1] <= img.shape[1]

        w, h = img.shape[:2]
        x, y = np.random.randint(h-crop_size[0]), np.random.randint(w-crop_size[1])

        img = img[y:y+crop_size[0], x:x+crop_size[1],:]
        img = skimage.transform.resize(img, size_img)

        label = label[y:y+crop_size[0], x:x+crop_size[1],:]
        label = skimage.transform.resize(label, size_label)
        return img, label

    def affine_transform(self, image, label, alpha_affine=0.5, random_state=None):

        if random_state is None:
            random_state = np.random.RandomState(None)

        shape = image.shape
        shape_size = shape[:2]
        center_square = np.float32(shape_size) // 2
        square_size = min(shape_size) // 3
        pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
        pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
        M = cv2.getAffineTransform(pts1, pts2)

        image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
        image = image[...,np.newaxis]
        label = cv2.warpAffine(label, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
        return image, label


    def data_augment(self, img, mask, chance=1):
        # flip l/r
        if random.uniform(0,1) < chance:
            img = cv2.flip( img, 1 )
            mask = cv2.flip( mask, 1 )
            if len(img.shape) == 2:
                img = img[...,np.newaxis]
            if len(mask.shape) == 2:
                mask = mask[...,np.newaxis]

        # random crop and resize
        if random.uniform(0,1) < chance:
            img, mask = self.random_crop_resize(img, mask)
            if len(img.shape) == 2:
                img = img[...,np.newaxis]
            if len(mask.shape) == 2:
                label = label[...,np.newaxis]

        # random affine transformation
        if random.uniform(0,1) < chance:
            img, mask = self.affine_transform(img, mask, alpha_affine=20)
            if len(img.shape) == 2:
                img = img[...,np.newaxis]
            if len(mask.shape) == 2:
                mask = mask[...,np.newaxis]
            
        # random gaussian noise
        if random.uniform(0,1) < chance:
            img = self.gaussian_noise(img)
        
        return img, mask

    def normalise(self, x, mean, std):
        return (x - mean) / std


    def read_array_list(self, arr_path_list):
        return np.array([np.load(arr_path) for arr_path in arr_path_list])
    
    
    def __getitem__(self, batch_index):
        # Get batch at index: batch_index

        # Handle case when not enough inputs for a full batch
        # NOTE this changes the values for the generator
        # Need to initialise again if triggered
        if (batch_index + 1) * self.batch_size > len(self.input_paths):
            batch_size = len(self.input_paths) - batch_index * self.batch_size
        else:
            batch_size = self.batch_size

        # Extract input paths for one batch at index: batch_index
        batch_input_paths = self.input_paths[ batch_index * batch_size : (batch_index + 1) * batch_size ] 
        batch_label_paths = self.label_paths[ batch_index * batch_size : (batch_index + 1) * batch_size ] 
        
        batch_imgs = []
        batch_masks = []
        
        for x, y in zip(batch_input_paths, batch_label_paths):
            x = np.load(x)
            y = np.load(y)
            if self.augment is True:
                x = self.normalise(x, self.training_mean, self.training_std)
                
                if random.uniform(0,1) < 0.5:
                    x, y = self.data_augment(x, y, chance=0.5)

            batch_imgs.append(x)
            batch_masks.append(y)
        
        return np.array(batch_imgs, dtype=np.float32), np.array(batch_masks, dtype=np.float32)
    

In [ ]:
data_mean = 168.3172158554484
data_std = 340.21625683608994
batch_size = 3


valid_gen = make_gen(valid_inputs,
                     valid_truths,
                     batch_size,
                     data_mean,
                     data_std, 
                     shuffle_on_end=False, 
                     augment=False)


train_gen = make_gen(train_inputs,
                     train_truths,
                     batch_size,
                     data_mean,
                     data_std, 
                     shuffle_on_end=True, 
                     augment=True)


test_gen = make_gen(test_inputs,
                     test_truths,
                     batch_size,
                     data_mean,
                     data_std, 
                     shuffle_on_end=False, 
                     augment=False)

In [ ]:
x, y = train_gen.__getitem__(0)
x.shape, y.shape

In [ ]:
import train as train
import tensorflow as tf

INITIAL_LR = 1e-4
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = tf.keras.losses.BinaryCrossentropy()
WEIGHTS = "./vacbag/initial.hdf5"



model = train.compile_model(1, OPTIMIZER, LOSS, weights=WEIGHTS)

In [ ]:
epochs = 10
steps_per_epoch = train_gen.__len__()
valid_steps = valid_gen.__len__()

print(steps_per_epoch)
print(valid_steps)

print("\n Training...")
train_history = model.fit(train_gen,
                          epochs=epochs,
                          steps_per_epoch=steps_per_epoch,
                          validation_steps=valid_steps,
                          validation_data=valid_gen,
                          verbose=1)

In [ ]:
model.save_weights("test_weights.hdf5")

In [ ]:
import loss as loss

INITIAL_LR = 1e-4
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = loss.dsc_loss
WEIGHTS = "test_weights.hdf5"



model = train.compile_model(1, OPTIMIZER, LOSS, weights=WEIGHTS)

In [ ]:
 w
    
    
    from datetime import datetime
    
    date_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    file_name = model_save + "ep_{epoch:02d}_vl{val_loss:.2f}_" + str(date_time)
    model_save = file_name + ".hdf5"
    log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=stop_patience, verbose=1, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=lr_scale, patience=lr_patience, verbose=1)
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_save, save_weights_only=True, verbose=1)
    csv_logger = tf.keras.callbacks.CSVLogger(file_name, separator=',', append=False)
    tensor_board = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                                 histogram_freq=5,
                                                                 write_graph=True,
                                                                 write_images=True,
                                                                 embeddings_freq=5,
                                                                 update_freq='epoch')
    
    callbacks = [early_stopping, model_checkpoint, reduce_lr, csv_logger, tensor_board]

In [ ]:
epochs = 1000
steps_per_epoch = train_gen.__len__()
valid_steps = valid_gen.__len__()

print(steps_per_epoch)
print(valid_steps)

print("\n Training...")
train_history = model.fit(train_gen,
                          epochs=epochs,
                          steps_per_epoch=steps_per_epoch,
                          validation_steps=valid_steps,
                          validation_data=valid_gen,
                          callbacks=callbacks,
                          verbose=1)

In [ ]:
model.load_weights("./vacbag/2_dice_after_bce_41_0.04_2020-04-29-16-21-27.hdf5_12_0.35_2020-04-29-18-54-51.hdf5")

In [ ]:
x, y = test_gen.__getitem__(0)
x.shape, y.shape

plt.plot()
plt.imshow(x[0,...,0])

In [ ]:
out = model.predict(x)
out.shape

In [ ]:
plt.imshow(out[1,...,0])